# House Prices: Advanced Regression Techniques

### AI TF 머신러닝 과제

* https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
* Predict sales prices and practice feature engineering, RFs, and gradient boosting

### File descriptions

* **input/train.csv** 테스트 셋
* **input/test.csv** 트레인 셋
* **input/data_description.txt** 데이터 설명
* **input/sample_sumbission.csv** 정답 제출 샘플

### XGBRegressor 모델 사용
* ** Kagle Score : 0.13536 **
* Linear Regression 에서 사용한 feature 들만 사용 시 결과가 더 안 좋음
* 결측치가 많은 feature 들만 제외하고 나머지 모든 feature 사용
* feature 정리를 조금 더 정밀하게 하면 점수가 좋아질것 같음 
  - 수치형 categorical 데이터를 string 으로
  - 제외시킬 feature 추가 검토

In [122]:
import pandas as pd
import numpy as np

## Load Dataset

In [123]:
# train data
train = pd.read_csv("input/train.csv", index_col="Id")

print(train.shape)
train.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [124]:
# test data
test = pd.read_csv("input/test.csv", index_col="Id")

print(test.shape)
test.head()

(1459, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Preprocessing

#### linear regression 에서 feature 를 선택하는 방법
1. 데이터 탐색을 통해 SalePrice 와 선형 상관관계가 높고, 선형 regression 전제조건을 충족하는 feature 선택
2. 모든 feature 를 사용하고 Backward Elimination 을 통해 feature 를 줄여나가는 방법

#### sklearn linear regression 에서 feature scaling 이 필요한가?
1. linear regression 에서는 자동으로 feature scaling 되는것으로 알고 있음
   - SVN, K-Mean, Logistic Regression 처럼 feature scaling 이 필요한 모델들도 있음
   - from sklearn.preprocessing import StandardScaler 사용
2. lenear regression 에서 feature scaling 을 했을경우와 안했을경우 차이를 확인해 보자
   - 차이가 없음 : 자동으로 feature scaling 되고 있음

### outlier

1. 'GrLiveArea' 의 outlier 데이터 제거 안함

In [125]:
#deleting points
# train.sort_values(by = 'GrLivArea', ascending = False)[:2]
# train = train.drop(train[train.index == 1299].index)
# train = train.drop(train[train.index == 524].index)
# print(train.shape)

In [126]:
# YearBuilt 에 공백값이 있는 데이터가 한건 있어서 임의의 값으로 업데이트
train.loc[train["YearBuilt"] == " ", "YearBuilt"] = 1980
# YearBuil 가 object type 이어서 수치형으로 변환
train['YearBuilt'] = pd.to_numeric(train['YearBuilt'])

### One hot encoding

Categorical 데이터를 모두 one hot encoding 처리하기 위한 작업

In [127]:
# one-hot encoding 시 train/test 데이터의 feature 들이 가지고 있는 값들이 다르면 dummy column 갯수에 차이가 날 수 있음
# 하나로 합쳐놓고 전처리 한 다음에 다시 나누도록..
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))
print(all_data.shape)
all_data.head()

(2919, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [128]:
# categorical 데이터를 one hot encoding
all_data_enc = pd.get_dummies(all_data)
print(all_data_enc.shape)
all_data_enc.head()

#all_data_enc.to_csv('features.csv', index=False)

(2919, 288)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [129]:
# train/predict 시 제외할 feature 지정
feature_drop = ['MasVnrArea','BsmtFinSF2','LowQualFinSF', 'LotFrontage','GarageYrBlt']

In [130]:
# train 시킬 데이터셋
X_train = all_data_enc[:train.shape[0]]
X_train = X_train.drop(feature_drop, axis=1)

print(X_train.shape)
X_train.tail()

(1460, 283)


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1456,60,7917,6,5,1999,2000,0.0,953.0,953.0,953,...,0,0,0,1,0,0,0,0,1,0
1457,20,13175,6,6,1978,1988,790.0,589.0,1542.0,2073,...,0,0,0,1,0,0,0,0,1,0
1458,70,9042,7,9,1941,2006,275.0,877.0,1152.0,1188,...,0,0,0,1,0,0,0,0,1,0
1459,20,9717,5,6,1950,1996,49.0,0.0,1078.0,1078,...,0,0,0,1,0,0,0,0,1,0
1460,20,9937,5,6,1965,1965,830.0,136.0,1256.0,1256,...,0,0,0,1,0,0,0,0,1,0


In [131]:
# test 데이터셋
X_test = all_data_enc[test.shape[0]+1:]
X_test = X_test.drop(feature_drop, axis=1)

print(X_test.shape)
X_test.head()

(1459, 283)


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,11622,5,6,1961,1961,468.0,270.0,882.0,896,...,0,0,0,1,0,0,0,0,1,0
1462,20,14267,6,6,1958,1958,923.0,406.0,1329.0,1329,...,0,0,0,1,0,0,0,0,1,0
1463,60,13830,5,5,1997,1998,791.0,137.0,928.0,928,...,0,0,0,1,0,0,0,0,1,0
1464,60,9978,6,6,1998,1998,602.0,324.0,926.0,926,...,0,0,0,1,0,0,0,0,1,0
1465,120,5005,8,5,1992,1992,263.0,1017.0,1280.0,1280,...,0,0,0,1,0,0,0,0,1,0


### missing data in X_train

In [132]:
#missing data
total = X_train.isnull().sum().sort_values(ascending=False)
percent = (X_train.isnull().sum()/X_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

,Total,Percent
SaleCondition_Partial,0,0.0
Condition2_PosN,0,0.0
Condition1_RRNe,0,0.0
Condition1_RRNn,0,0.0
Condition2_Artery,0,0.0


### missing data in X_test

In [133]:
#missing data
total = X_test.isnull().sum().sort_values(ascending=False)
percent = (X_test.isnull().sum()/X_test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

,Total,Percent
BsmtFullBath,2,0.001371
BsmtHalfBath,2,0.001371
GarageArea,1,0.000685
GarageCars,1,0.000685
BsmtFinSF1,1,0.000685
BsmtUnfSF,1,0.000685
TotalBsmtSF,1,0.000685
Condition1_RRNn,0,0.000000
Condition2_Artery,0,0.000000
Condition2_Feedr,0,0.000000


In [135]:
# TotalBsmtSF null 값 확인
X_test[X_test['TotalBsmtSF'].isnull()]

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
2121,20,5940,4,7,1946,1950,NaN,NaN,NaN,896,...,0,0,0,0,1,0,0,0,0,0


In [136]:
# 결측치를 평균값으로 채움
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test["TotalBsmtSF"]=imp.fit_transform(X_test[["TotalBsmtSF"]]).ravel()

# total_bsmtsf_mean = X_test['TotalBsmtSF'].mean()
# X_test.loc[X_test['TotalBsmtSF'].isnull(), 'TotalBsmtSF'] = total_bsmtsf_mean

In [137]:
# GarageCars null 값 확인
X_test[X_test['GarageCars'].isnull()]

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
2577,70,9060,5,6,1923,1999,548.0,311.0,859.0,942,...,0,0,0,1,0,0,1,0,0,0


In [138]:
# 결측치를 평균값으로 채움
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test["GarageCars"]=imp.fit_transform(X_test[["GarageCars"]]).ravel()

# garagecars_mean = X_test['GarageCars'].mean()
# X_test.loc[X_test['GarageCars'].isnull(), 'GarageCars'] = garagecars_mean

### feature scaling

RandomForestRegressor 는 feature scaling 이 필요 없음

In [139]:
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)

### model 

In [140]:
# train 시킬 때 사용할 label(target, 종속변수) 컬럼 선택
label_name = "SalePrice"

# train 시킬 때 사용할 label(target, 종속변수) 데이터셋 준비
y_train = np.log(train[label_name])

print(y_train.shape)
y_train.head()

(1460,)


Id
1    12.247694
2    12.109011
3    12.317167
4    11.849398
5    12.429216
Name: SalePrice, dtype: float64

In [155]:
from xgboost.sklearn import XGBRegressor
model = XGBRegressor(n_estimators=300,
                     max_depth=10,
                     seed=37,
                     learning_rate=0.05);
model

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=37, silent=True, subsample=1)

### Scoring

Kaggle 에서 log 함수로 변환된 SalePrice 값을 RMSE 방식으로 scoring 하기 때문에 RMSE 함수 구현하여 사용

In [156]:
# RMSE 함수 구현
import numpy as np
from sklearn.metrics import make_scorer

def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)
  
    difference = predict - actual
    difference = np.square(difference)
    
    mean_difference = difference.mean()
    
    score = np.sqrt(mean_difference)
    
    return score

rmse_scorer = make_scorer(rmse)
rmse_scorer

make_scorer(rmse)

In [157]:
from sklearn.cross_validation import cross_val_score

score = cross_val_score(model, X_train, y_train, cv=10, \
                         scoring=rmse_scorer).mean()

print("Score = {0:.5f}".format(score))

Score = 0.12680


## Hyperparameter Tuning
https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

In [ ]:
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.cross_validation import cross_val_score

hyperparameters_list = []

n_estimators = 300
num_epoch = 10

for epoch in range(num_epoch):
    max_depth = np.random.randint(low=5, high=20)
    learning_rate = np.random.uniform(low=0.01, high=0.1)

    model = XGBRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  learning_rate=learning_rate,
                                  seed=37)

    score = cross_val_score(model, X_train, y_train, cv=10, \
                            scoring=rmse_scorer).mean()

    hyperparameters_list.append({
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'learning_rate': learning_rate,
    })

    print("Learning Rate = {0: .3f} Score = {1:.5f}".format(learning_rate, score))
 
hyperparameters_list = pd.DataFrame.from_dict(hyperparameters_list)
hyperparameters_list = hyperparameters_list.sort_values(by="score")

print(hyperparameters_list.shape)
hyperparameters_list.head()

## Train

In [144]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.03, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=37, silent=True, subsample=1)

## Predict

In [145]:
# RMSE scoreing 하기 위해 log로 변환했던 값을 다시 원복해야 함
predictions = np.exp(model.predict(X_test))
print(predictions.shape)
predictions

(1459,)


array([ 121383.8046875,  161932.0625   ,  183866.359375 , ...,
        157437.15625  ,  120033.0390625,  232570.765625 ], dtype=float32)

## Submit (kaggle 제출용)

In [146]:
submission = pd.read_csv("input/sample_submission.csv", index_col="Id")

submission["SalePrice"] = predictions

print(submission.shape)
submission.head()

(1459, 1)


,SalePrice
Id,
1461,121383.804688
1462,161932.062500
1463,183866.359375
1464,185005.609375
1465,189291.953125


In [148]:
# 저장할 파일을 구분하기 위해 파일명에 timestamp 정보 추가 하기 위한 작업 
from datetime import datetime

current_date = datetime.now()
current_date = current_date.strftime("%Y-%m-%d_%H-%M-%S")

description = "xgb-regressor"

filename = "{date}_{desc}_{score}.csv".format(date=current_date, desc=description, score="{0:.5f}".format(score))
filepath = "output/{filename}".format(filename=filename)

submission.to_csv(filepath)